In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
pd.set_option('display.max_colwidth', None)
from time import sleep
import datetime
import re


In [2]:
# Initialize an empty list to store medium search topics
medium_search_topics = []

# Create an empty DataFrame with columns 'url' and 'text'
df = pd.DataFrame(columns=['url', 'text'])

# Display the empty DataFrame
df


,url,text


In [3]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to search for articles on Medium
def search_medium(query):
    # Initialize an empty list to store article URLs
    href_list = []
    
    # Construct the URL for Medium search
    url = f"https://medium.com/search/publications?q={query}"
    
    # Send a GET request to the search URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the search page
        page_content = response.text
        soup = BeautifulSoup(page_content, "html.parser")
        
        # Find all <a> tags on the page
        all_a_tags = soup.find_all('a')

        # Iterate over <a> tags to extract article information
        for a_tag in all_a_tags:
            # Find <h2> tag within the <a> tag
            h2_tag = a_tag.find('h2')
            
            # Check if <h2> tag exists (indicating an article)
            if h2_tag is not None:
                h2_text = h2_tag.get_text().strip()  # Extract article title
                href_attribute = a_tag.get('href')   # Get the 'href' attribute
                full_url = urljoin(url, href_attribute)  # Convert relative path to full URL
                href_list.append(full_url)  # Append the full URL to the list
    
    return href_list

# Function to extract article information from given URLs
def extract_article_info(href_list):
    # Initialize an empty list to store article URLs
    article_href = []

    # Iterate over URLs in the input list
    for publication in href_list:
        url = publication
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the article page
            page_content = response.text
            soup = BeautifulSoup(page_content, "html.parser")
            
            # Find all <a> tags on the page
            all_a_tags = soup.find_all('a')

            # Iterate over <a> tags to extract article information
            for a_tag in all_a_tags:
                # Find <h2> tag within the <a> tag
                h2_tag = a_tag.find('h2')
                
                # Check if <h2> tag exists (indicating an article)
                if h2_tag is not None:
                    h2_text = h2_tag.get_text().strip()  # Extract article title
                    href_attribute = a_tag.get('href')   # Get the 'href' attribute
                    full_url = urljoin(url, href_attribute)  # Convert relative path to full URL
                    article_href.append(full_url)  # Append the full URL to the list
    
    return article_href

# Function to scrape text from Medium articles
def scrape_medium_articles(article_href, df, min_p_tags=3):
    # Initialize a list to store new data
    new_data = []

    # Iterate over URLs in the input list
    for id, url in enumerate(article_href):
        # Send a GET request to the article URL
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content of the article page
            page_content = response.text
            soup = BeautifulSoup(page_content, "html.parser")

            # Find all <p> tags with class containing "pw-post"
            p_tags = soup.find_all('p', class_=lambda c: c and 'pw-post' in c)

            # Check if the article has at least `min_p_tags` <p> tags
            if len(p_tags) >= min_p_tags:
                text = ""

                # Iterate over <p> tags to extract article text
                for p_tag in p_tags:
                    # Exclude <p> tags starting with "By"
                    if not p_tag.get_text().strip().startswith("By"):
                        text += p_tag.get_text() + " "

                new_data.append([url, text])
    
    # Check if new data is available
    if new_data:
        # Create a new DataFrame from the new data
        new_df = pd.DataFrame(new_data, columns=['url', 'text'])
        # Append the new data to the existing DataFrame
        df = pd.concat([df, new_df], ignore_index=True)
    
    return df


In [ ]:
# List of topics to search for on Medium
search_list = ['data science', 'running', 'space exploration', 'crypto', 'finance', 'machine learning',
               'investment', 'entertainment', 'health', 'politics', 'self improvement', 'travel', 'sports',
               'business', 'pollution', 'gaming', 'psychology', 'parenting', 'career development',
               'entrepreneurship', 'space', 'language', 'olympics']

# List to store searched topics
searched = []

# Loop through each search topic in the list
for search in search_list:
    # Search for articles on Medium related to the current topic
    href_list = search_medium(search)
    
    # Extract article information from the search results
    article_href = extract_article_info(href_list)
    
    # Scrape text from Medium articles and update the DataFrame
    df = scrape_medium_articles(article_href, df)
    
    # Append the current search topic to the list of searched topics
    searched.append(search)
    
    # Print the list of searched topics
    print(searched)
    
    # Save the updated DataFrame to a CSV file
    df.to_csv('text_summarization_data.csv', index=False)



In [5]:
# Display the first 5 rows showing the articles
df.head()

,url,text
0,https://towardsdatascience.com/exploratory-data-analysis-unraveling-the-story-within-your-dataset-6a8b1acdde?source=author_recirc-----b6482de97d2e----3---------------------d269713c_0d48_41b5_8771_a9b80c3e5ebf-------,"As a data enthusiast, exploring a new dataset is an exciting endeavour. It allows us to gain a deeper understanding of the data and lays the foundation for successful analysis. Getting a good feeling for a new dataset is not always easy, and takes time. However, a good and thorough exploratory data analysis (EDA) can help a lot to understand your dataset and get a feeling for how things are connected and what needs to be done to properly process your dataset. Infact, you probably will spend 80% of your time in data preparation and exploration and only 20% in actual data modelling. For other types of analysis, exploration might take an even larger proportion of your time. Exploratory Data Analysis, simply put, refers to the art of exploring data. It is the process of investigating data from different angles to enhance your understanding, exploring patterns, establishing relationships between variables and if required enhancing the data itself Its like going on a ‘blind’ date with your dataset, sitting across the table from this enigmatic collection of numbers and texts, yearning to understand it before embarking on a serious relationship. Just like a blind date, EDA allows you to uncover the hidden facets of your dataset. You observe patterns, detect outliers, and explore the nuances before making any significant commitments. It’s all about getting acquainted and building trust with the numbers, ensuring you’re on solid ground before drawing conclusions. We as analysts and data scientists are supposed to best understand the data. We must become the experts when it comes to understanding and interpreting the data. Whether it is machine learning models, experimentation frameworks or simple analytics — the outcome is as good as the data on which it is based."
1,https://medium.com/@ignacio.de.gregorio.noblejas/openai-just-killed-an-entire-market-in-45-minutes-818b2a8ad33e?source=read_next_recirc-----b6482de97d2e----0---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,"This is a story about how a 45-minute speech killed an entire billion-dollar market. This week OpenAI had its first-ever developer conference. Fewer times in history had there been higher expectations of a private technology event, and the OpenAI did not disappoint. They announced a new, smarter GPT-4 model named ‘Turbo’ and they included several fascinating features that severely improve outputs and offer developers much more control over their models. And all this while still managing to decrease, once again, API costs. But one announcement went swiftly past most of the media while being the one with the highest immediate impact on the AI market, as this new feature alone threatens an entire start-up ecosystem and billions of invested capital. Even OpenAI couldn’t have predicted it. Not even they saw ChatGPT coming. Over the last few months, we have seen multiple accounts from the top management on OpenAI like Ilya Sutskever, Mina Murati, and Sam Altman claim that when they released the open beta for ChatGPT back in November 2022 they didn’t expect it to have that huge access. In two months, it had already reached 100 million first-time users, and was already officially the fastest-growing…"
2,https://medium.com/speaking-bipolar/bipolar-and-exercise-f9a1a7f9a9c1?source=read_next_recirc-----b6482de97d2e----1---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,"Exercise is often recommended for mental well-being, but in my experience, intense activities like hurling and running can sometimes make my mania symptoms worse. I used to play hurling, but now I stick to running because I enjoy the feeling of completing a run. Looking back two years, I made a risky choice to stop taking my medication and started running four t

## Data Cleaning

Removing all special characters, punctuation, and unnecesary white spaces.

In [6]:
def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra white spaces
    text = ' '.join(text.split())

    return text


In [7]:
# Cleaning the text
df['text'] = df['text'].apply(clean_text)

In [8]:
# Display the first 5 cleaned articles
df.head()

,url,text
0,https://towardsdatascience.com/exploratory-data-analysis-unraveling-the-story-within-your-dataset-6a8b1acdde?source=author_recirc-----b6482de97d2e----3---------------------d269713c_0d48_41b5_8771_a9b80c3e5ebf-------,As a data enthusiast exploring a new dataset is an exciting endeavour It allows us to gain a deeper understanding of the data and lays the foundation for successful analysis Getting a good feeling for a new dataset is not always easy and takes time However a good and thorough exploratory data analysis EDA can help a lot to understand your dataset and get a feeling for how things are connected and what needs to be done to properly process your dataset Infact you probably will spend 80 of your time in data preparation and exploration and only 20 in actual data modelling For other types of analysis exploration might take an even larger proportion of your time Exploratory Data Analysis simply put refers to the art of exploring data It is the process of investigating data from different angles to enhance your understanding exploring patterns establishing relationships between variables and if required enhancing the data itself Its like going on a blind date with your dataset sitting across the table from this enigmatic collection of numbers and texts yearning to understand it before embarking on a serious relationship Just like a blind date EDA allows you to uncover the hidden facets of your dataset You observe patterns detect outliers and explore the nuances before making any significant commitments Its all about getting acquainted and building trust with the numbers ensuring youre on solid ground before drawing conclusions We as analysts and data scientists are supposed to best understand the data We must become the experts when it comes to understanding and interpreting the data Whether it is machine learning models experimentation frameworks or simple analytics the outcome is as good as the data on which it is based
1,https://medium.com/@ignacio.de.gregorio.noblejas/openai-just-killed-an-entire-market-in-45-minutes-818b2a8ad33e?source=read_next_recirc-----b6482de97d2e----0---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,This is a story about how a 45minute speech killed an entire billiondollar market This week OpenAI had its firstever developer conference Fewer times in history had there been higher expectations of a private technology event and the OpenAI did not disappoint They announced a new smarter GPT4 model named Turbo and they included several fascinating features that severely improve outputs and offer developers much more control over their models And all this while still managing to decrease once again API costs But one announcement went swiftly past most of the media while being the one with the highest immediate impact on the AI market as this new feature alone threatens an entire startup ecosystem and billions of invested capital Even OpenAI couldnt have predicted it Not even they saw ChatGPT coming Over the last few months we have seen multiple accounts from the top management on OpenAI like Ilya Sutskever Mina Murati and Sam Altman claim that when they released the open beta for ChatGPT back in November 2022 they didnt expect it to have that huge access In two months it had already reached 100 million firsttime users and was already officially the fastestgrowing
2,https://medium.com/speaking-bipolar/bipolar-and-exercise-f9a1a7f9a9c1?source=read_next_recirc-----b6482de97d2e----1---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,Exercise is often recommended for mental wellbeing but in my experience intense activities like hurling and running can sometimes make my mania symptoms worse I used to play hurling but now I stick to running because I enjoy the feeling of completing a run Looking back two years I made a risky choice to stop taking my medication and started running four times a week I even signed up for the Clontarf half marathon aiming to finish in und

In [9]:
# Import the necessary libraries
from transformers import pipeline

In [10]:
# Create a summarization pipeline using the pre-trained model
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [11]:
# Add a new column 'length' to the DataFrame based on the length of the 'text' column
df['length'] = df['text'].apply(len)

In [12]:
# Filter the DataFrame to include only rows where the length of 'text' is between 1000 and 5500 characters
# This is due to the pre-trained model only being able to handle certain lengths
df = df[(df.length < 5500) & (df.length > 1000)]

In [13]:
# Create a new empty column named 'summary' in the DataFrame
df['summary'] = ''

# Display the first few rows of the DataFrame to show the addition of the new 'summary' column
df.head()

,url,text,length,summary
0,https://towardsdatascience.com/exploratory-data-analysis-unraveling-the-story-within-your-dataset-6a8b1acdde?source=author_recirc-----b6482de97d2e----3---------------------d269713c_0d48_41b5_8771_a9b80c3e5ebf-------,As a data enthusiast exploring a new dataset is an exciting endeavour It allows us to gain a deeper understanding of the data and lays the foundation for successful analysis Getting a good feeling for a new dataset is not always easy and takes time However a good and thorough exploratory data analysis EDA can help a lot to understand your dataset and get a feeling for how things are connected and what needs to be done to properly process your dataset Infact you probably will spend 80 of your time in data preparation and exploration and only 20 in actual data modelling For other types of analysis exploration might take an even larger proportion of your time Exploratory Data Analysis simply put refers to the art of exploring data It is the process of investigating data from different angles to enhance your understanding exploring patterns establishing relationships between variables and if required enhancing the data itself Its like going on a blind date with your dataset sitting across the table from this enigmatic collection of numbers and texts yearning to understand it before embarking on a serious relationship Just like a blind date EDA allows you to uncover the hidden facets of your dataset You observe patterns detect outliers and explore the nuances before making any significant commitments Its all about getting acquainted and building trust with the numbers ensuring youre on solid ground before drawing conclusions We as analysts and data scientists are supposed to best understand the data We must become the experts when it comes to understanding and interpreting the data Whether it is machine learning models experimentation frameworks or simple analytics the outcome is as good as the data on which it is based,1744,
1,https://medium.com/@ignacio.de.gregorio.noblejas/openai-just-killed-an-entire-market-in-45-minutes-818b2a8ad33e?source=read_next_recirc-----b6482de97d2e----0---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,This is a story about how a 45minute speech killed an entire billiondollar market This week OpenAI had its firstever developer conference Fewer times in history had there been higher expectations of a private technology event and the OpenAI did not disappoint They announced a new smarter GPT4 model named Turbo and they included several fascinating features that severely improve outputs and offer developers much more control over their models And all this while still managing to decrease once again API costs But one announcement went swiftly past most of the media while being the one with the highest immediate impact on the AI market as this new feature alone threatens an entire startup ecosystem and billions of invested capital Even OpenAI couldnt have predicted it Not even they saw ChatGPT coming Over the last few months we have seen multiple accounts from the top management on OpenAI like Ilya Sutskever Mina Murati and Sam Altman claim that when they released the open beta for ChatGPT back in November 2022 they didnt expect it to have that huge access In two months it had already reached 100 million firsttime users and was already officially the fastestgrowing,1180,
2,https://medium.com/speaking-bipolar/bipolar-and-exercise-f9a1a7f9a9c1?source=read_next_recirc-----b6482de97d2e----1---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,Exercise is often recommended for mental wellbeing but in my experience intense activities like hurling and running can sometimes make my mania symptoms worse I used to play hurling but now I stick to running because I enjoy the feeling of completing a run Looking back two years I made a risky choice to stop taking my medication and started running four times a week I even signed up for the Clontarf half marat

In [ ]:
# Define a function to generate summaries using the pre-trained summarization model
def summarize_text(x):
    try:
        # Use the summarization pipeline to generate a summary for the input text
        result = summarizer(x, max_length=100, min_length=30, do_sample=False)
        # Extract the generated summary from the result
        summary_text = result[0]['summary_text']
    except Exception as e:
        # Handle exceptions, print an error message, and set the summary_text to an empty string
        print(f"Error summarizing text: {str(e)}")
        summary_text = ""
    return summary_text

# Apply the summarize_text function to each entry in the 'text' column and store the results in the 'summary' column
df['summary'] = df['text'].apply(summarize_text)



In [15]:
# Write the dataframe with summaries to a csv file
df.to_csv('summarized_text.csv')

In [16]:
# View the top 5 rows with their texts and summaries
df.head()

,url,text,length,summary
0,https://towardsdatascience.com/exploratory-data-analysis-unraveling-the-story-within-your-dataset-6a8b1acdde?source=author_recirc-----b6482de97d2e----3---------------------d269713c_0d48_41b5_8771_a9b80c3e5ebf-------,As a data enthusiast exploring a new dataset is an exciting endeavour It allows us to gain a deeper understanding of the data and lays the foundation for successful analysis Getting a good feeling for a new dataset is not always easy and takes time However a good and thorough exploratory data analysis EDA can help a lot to understand your dataset and get a feeling for how things are connected and what needs to be done to properly process your dataset Infact you probably will spend 80 of your time in data preparation and exploration and only 20 in actual data modelling For other types of analysis exploration might take an even larger proportion of your time Exploratory Data Analysis simply put refers to the art of exploring data It is the process of investigating data from different angles to enhance your understanding exploring patterns establishing relationships between variables and if required enhancing the data itself Its like going on a blind date with your dataset sitting across the table from this enigmatic collection of numbers and texts yearning to understand it before embarking on a serious relationship Just like a blind date EDA allows you to uncover the hidden facets of your dataset You observe patterns detect outliers and explore the nuances before making any significant commitments Its all about getting acquainted and building trust with the numbers ensuring youre on solid ground before drawing conclusions We as analysts and data scientists are supposed to best understand the data We must become the experts when it comes to understanding and interpreting the data Whether it is machine learning models experimentation frameworks or simple analytics the outcome is as good as the data on which it is based,1744,Exploratory Data Analysis simply put refers to the art of exploring data It is the process of investigating data from different angles to enhance your understanding exploring patterns establishing relationships between variables and if required enhancing the data itself . The outcome is as good as the data on which it is based .
1,https://medium.com/@ignacio.de.gregorio.noblejas/openai-just-killed-an-entire-market-in-45-minutes-818b2a8ad33e?source=read_next_recirc-----b6482de97d2e----0---------------------48d99472_be32_4463_88ac_dbc34a1255b6-------,This is a story about how a 45minute speech killed an entire billiondollar market This week OpenAI had its firstever developer conference Fewer times in history had there been higher expectations of a private technology event and the OpenAI did not disappoint They announced a new smarter GPT4 model named Turbo and they included several fascinating features that severely improve outputs and offer developers much more control over their models And all this while still managing to decrease once again API costs But one announcement went swiftly past most of the media while being the one with the highest immediate impact on the AI market as this new feature alone threatens an entire startup ecosystem and billions of invested capital Even OpenAI couldnt have predicted it Not even they saw ChatGPT coming Over the last few months we have seen multiple accounts from the top management on OpenAI like Ilya Sutskever Mina Murati and Sam Altman claim that when they released the open beta for ChatGPT back in November 2022 they didnt expect it to have that huge access In two months it had already reached 100 million firsttime users and was already officially the fastestgrowing,1180,This is a story about how a 45minute speech killed an entire billiondollar market . OpenAI announced a new smarter GPT4 model named Turbo and they included several fascinating features that severely improve outputs and offer developers much more control over their m